# Video frame and timestamp matching

The actual video recording starts a few seconds after the logging data's video recording start timing (the second the command was issued). This is because it takes some time for the logger's camera module to start actual video recording after receiving the command. This delay after sending the video recording start command must be considered to adjust the timestamp.

In [ ]:
import os
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import sys
sys.path.append("../") # Set parent directory to sys.path
sys.dont_write_bytecode = True
%load_ext autoreload
%autoreload 2
import src.utils as utils

palette5 = sns.color_palette(["#888888", "#D81B60", "#1E88E5", "#FFC107", "#004D40"])
palette2 = sns.color_palette(["#D81B60", "#1E88E5", "#FFC107", "#004D40"])
palette = palette2
display(palette)
sns.set_theme(context='poster', style='ticks', palette=palette, font_scale=1.0)

## Config

In [ ]:
FPS = 30
REC_SEC = 185
REC_SEC_SHORT = 126

# Shift mode
# shift_mode = 0 # no shift
shift_mode = 1 # simply shift by the amount of frame_diff

OFFSET = 3 # 3 ( < 3.6) frames, 99.99 ms which is close to 120 ms (200 - 80 ms)
EXP_TOTAL_FRAMES = FPS * REC_SEC + OFFSET
EXP_TOTAL_FRAMES_SHORT = FPS * REC_SEC_SHORT + OFFSET
START_SHIFT_ROWS = 0

# OFFSET = 4 # 4 ( > 3.6) frames, 133.32 ms which is close to 120 ms (200 - 80 ms)
# EXP_TOTAL_FRAMES = FPS * REC_SEC + OFFSET
# EXP_TOTAL_FRAMES_SHORT = FPS * REC_SEC_SHORT + OFFSET
# START_SHIFT_ROWS = 0

target_path = "../data/umineko-2024-v8i-yolov8/predicted-data/*.csv"
path_list = sorted(glob.glob(target_path))
print(f"len(path_list): {len(path_list)}")

## Add timestamp to 30 FPS frame data

In [ ]:
start_shift_rows = START_SHIFT_ROWS

path_list = sorted(glob.glob(target_path))
# print(len(path_list))

for i, path in enumerate(path_list):
# for i, path in enumerate(path_list[0:7]):
# for i, path in enumerate(path_list[7:8]): # LBP03 S00 for debug
    df_frames = pd.read_csv(path)
    test_id = os.path.basename(path)[:5]
    session_id = os.path.basename(path).replace(".csv", "")
    print(f"{test_id} | {session_id}")
    if session_id in ["LBP01_S03"]:
        continue # skip
    gps_data_path = f"../data/extracted-gps-data/{test_id}/{session_id}.csv"
    # print(gps_data_path)
    df_gps = pd.read_csv(gps_data_path)
    gps_colnames = [
        'datetime_jst', 'speed_distance_km_h', 'abs_diff_distance_m', 
        'camera_command', 'camera_recording', 'speaker_on'
    ]
    df_gps = df_gps[gps_colnames]
    # print(len(df_frames))
    # print(len(df_frames)/30)
    # print(len(df_gps))
    # display(df_gps.head(2))
    # display(df_gps.tail(2))

    # 1 Hz timestamp to 30 FPS timestamp
    df_gps_with_empty_rows = utils.insert_empty_rows(df_gps, FPS - 1)
    # print(len(df_gps_with_empty_rows))
    # display(df_gps_with_empty_rows.head(32))
    df_gps_filled = df_gps_with_empty_rows.ffill()
    # display(df_gps_filled.head(32))
    
    # adjust timestamp
    frame_diff = EXP_TOTAL_FRAMES - len(df_frames)
    if session_id in ["LBP06_S04"]:
        frame_diff = 78 # use the median value for LBP06 S04 (see fig_s07_frame_diff_test.ipynb)
    delay_ms = frame_diff / 30 * 1000
    delay_s = delay_ms / 1000
    print(f"EXP_TOTAL_FRAMES: {EXP_TOTAL_FRAMES}")
    print(f"frame diff: {frame_diff}")
    print(f"delay based on frame diff: {delay_ms:.2f} (ms)")
    print(f"delay based on frame diff: {delay_s:.2f} (s)")
    print(f"len(df_gps_filled): {len(df_gps_filled)}") # should be 5550 (185 sec * 30 FPS)

    df_gps_filled = df_gps_filled[start_shift_rows:]
    print(f"len(df_gps_filled): {len(df_gps_filled)}")
    idx = utils.get_speaker_turn_on_idx(df_gps_filled)
    print(f"idx: {idx}")

    if shift_mode == 1:
        _df_gps = df_gps_filled[frame_diff:]
        # print(len(_df_gps))
    _df_gps.reset_index(drop=True, inplace=True)
    print(f"len(df_frames): {len(df_frames)}")
    print(f"len(_df_gps): {len(_df_gps)}") 

    # Adjust the dataframe length | data near the end of recording are not needed for data analysis
    if len(_df_gps) > len(df_frames):
        _df_gps = _df_gps[:len(df_frames)]
    
    if len(df_frames) > len(_df_gps):
        df_frames = df_frames[:len(_df_gps)]

    print(f"len(df_frames): {len(df_frames)}")
    print(f"len(_df_gps): {len(_df_gps)}")

    # rename columns | absdiff -> abs_diff
    # NOTE: original column names 
    # ['pixel_count', 'absdiff_pixel_count', 'pixel_count_p', 'absdiff_pixel_count_p']
    df_frames.columns = [
        'pixel_count', 'abs_diff_pixel_count',
        'pixel_count_p', 'abs_diff_pixel_count_p'
    ]

    # Concat dataframes
    df = pd.concat([df_frames, _df_gps], axis=1)
    df.reset_index(drop=True, inplace=True)
    idx = utils.get_speaker_turn_on_idx(df)
    print(f"idx: {idx}")
    program_index = np.arange(1, len(df)+1, 1)
    _program_index = program_index - idx -1
    df.insert(0, 'program_index', program_index)
    df.insert(0, '_program_index', _program_index)
    display(df[idx-3:idx+3])
    
    if shift_mode == 1:
        save_dir = f"../data/umineko-2024-v8i-yolov8/offset-{OFFSET:02d}-start-shift-{start_shift_rows}"
    else:
        save_dir = f"../data/umineko-2024-v8i-yolov8/no-shift"
    os.makedirs(save_dir, exist_ok=True)
    save_path = f"{save_dir}/{session_id}.csv"
    df.to_csv(save_path, index=False)

## Shift simulation

In [ ]:
# Conduct an experiment with an extreme example

fps = 30
recording_sec = 10

start_delay_sec = 80 / 1000
stop_delay_sec = 200 / 1000

total_recording_sec = recording_sec - start_delay_sec + stop_delay_sec
print(total_recording_sec)

expected_total_frames = int(fps * total_recording_sec)
print(expected_total_frames) # 303 frames

In [ ]:
# Assume there is exactly a 2-second delay before the video recording starts
delay_sec = 2
delay_frames = int(fps * delay_sec)
print(delay_frames)
actual_total_frames = expected_total_frames - delay_frames
print(actual_total_frames)

In [ ]:
video_rec_indices_0 = np.arange(0, expected_total_frames, 1) # without delay
video_rec_indices_1 = np.arange(0, expected_total_frames - delay_frames, 1) # with delay
simulated_data = np.zeros(len(video_rec_indices_1))
intervention_index = 150 # true playback timing
spike_index_0 = intervention_index + int(FPS/2) # spike shortly after the playback
spike_index_1 = spike_index_0 - delay_frames
simulated_data[spike_index_1] = 1
print(len(video_rec_indices_0))
print(len(video_rec_indices_1))

### w/o shifting

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(12, 3))
ax.plot(video_rec_indices_1, simulated_data)
ax.axvline(x=intervention_index, color="#333333", linestyle="--")
ax.axvspan(xmin=intervention_index, xmax=intervention_index+30, color="#333333", alpha=0.1)
ax.set_yticks(np.arange(-1, 2, 1))
ax.set_xlim(0 -10, 300 + 10)
ax.set_ylim(-0.2, 1.2)
ax.grid()

### w/ shiting

In [ ]:
video_rec_indices_1_shifted = video_rec_indices_1 + delay_frames
print(video_rec_indices_1_shifted)
video_rec_indices_1_shifted = video_rec_indices_0[delay_frames:]
print(video_rec_indices_1_shifted)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(12, 3))
ax.plot(video_rec_indices_1_shifted, simulated_data)
ax.axvline(x=intervention_index, color="#333333", linestyle="--")
ax.axvspan(xmin=intervention_index, xmax=intervention_index+30, color="#333333", alpha=0.1)
ax.set_yticks(np.arange(-1, 2, 1))
ax.set_xlim(0 -10, 300 + 10)
ax.set_ylim(-0.2, 1.2)
ax.grid()